In [1]:
!pip install openai observers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.3/161.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Created wheel for standardwebhooks: filename=standardwebhooks-1.0.0-py3-none-any.whl size=3519 sha256=89ed65f41976b0257579c83fd75c07a21d2a7388b75f75a96fa482f45eb78821
  Stored in directory: /root/.cache/pip/wheels/7a/ad/74/53714a689cdbc9a47f2a581ffc834940461fab236ba87976c9
Successfully built standardwebhooks
  Attempting uninstall: fsspec
    Found existing installation: fss

In [15]:
import duckdb
import uuid
from datetime import datetime
from observers.observers import wrap_openai
from openai import OpenAI

# Connect to a local DuckDB instance (it will create a file-based DB if not exists)
db_path = "chat_logs.duckdb"
conn = duckdb.connect(database=db_path)

# Set up a table to log interactions
conn.execute("""
CREATE TABLE IF NOT EXISTS chat_logs (
    id TEXT, -- Store UUID as a text string
    model_name TEXT,
    role TEXT,
    content TEXT,
    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# Initialize Ollama client running locally; or use ngrok to spoof LM Studio, etc
openai_client = OpenAI(api_key="dummy_key", base_url="https://ee74-2600-1700-e4f0-525f-ee-71da-1b56-35e3.ngrok-free.app/v1/")
client = wrap_openai(openai_client)

# Function to log the chat messages to DuckDB
def log_to_duckdb(model_name, role, content):
    unique_id = str(uuid.uuid4())  # Generate UUID in Python
    current_time = datetime.now()  # Get the current timestamp
    conn.execute("""
    INSERT INTO chat_logs (id, model_name, role, content, timestamp)
    VALUES (?, ?, ?, ?, ?)
    """, (unique_id, model_name, role, content, current_time))

# Perform a chat completion with logging
try:
    model_name = "llama3.1"
    user_message = {"role": "user", "content": "Tell me a joke."}

    # Log user message
    log_to_duckdb(model_name, user_message["role"], user_message["content"])

    # Get response from the wrapped Ollama client
    response = client.chat.completions.create(
        model=model_name,
        messages=[user_message]
    )

    # Log assistant response
    assistant_message = response['choices'][0]['message']
    log_to_duckdb(model_name, assistant_message["role"], assistant_message["content"])

    print(assistant_message["content"])

except Exception as e:
    print(f"Error during chat completion: {e}")
finally:
    conn.close()


Error during chat completion: 'ChatCompletion' object is not subscriptable
